# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,16.40,51,100,1.34,AU,1697081126
1,1,tiksi,71.6872,128.8694,-4.30,82,100,7.08,RU,1697080877
2,2,port elizabeth,-33.9180,25.5701,18.16,96,100,3.60,ZA,1697081126
3,3,aberdeen,57.1437,-2.0981,7.03,91,47,1.03,GB,1697080929
4,4,ushuaia,-54.8000,-68.3000,4.81,75,40,12.07,AR,1697081109


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    scale = 0.65,
    color = 'City')
# Display the map
map1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [40]:
# Narrow down cities that fit criteria and drop any results with null values
idealCitytemp = city_data_df.drop(city_data_df[(city_data_df['Max Temp'] <= 15) & (city_data_df['Max Temp'] <= 28)].index, inplace=False)

# Drop any rows with null values
idealCitytemp.dropna(how='any', inplace=True)

# Display sample data
idealCitytemp.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,16.40,51,100,1.34,AU,1697081126
2,2,port elizabeth,-33.9180,25.5701,18.16,96,100,3.60,ZA,1697081126
8,8,rubio,7.7013,-72.3557,26.47,84,14,1.73,VE,1697081110
10,10,puerto santa,-8.9906,-78.6456,18.43,89,100,5.57,PE,1697081111
11,11,crane,31.3974,-102.3501,22.55,64,0,11.40,US,1697081111
15,15,adamstown,-25.0660,-130.1015,20.61,72,2,4.67,PN,1697080880
18,18,tofol,5.3248,163.0078,28.76,73,98,3.69,FM,1697081130
19,19,hawaiian paradise park,19.5933,-154.9731,25.02,87,75,5.01,US,1697081130
21,21,twon-brass,4.3151,6.2418,26.40,85,99,3.10,NG,1697080882
22,22,west island,-12.1568,96.8225,27.99,74,75,5.66,CC,1697081131


### Step 3: Create a new DataFrame called `hotel_df`.

In [47]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealCitytemp[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,blackmans bay,AU,-43.0167,147.3167,51,
2,port elizabeth,ZA,-33.9180,25.5701,96,
8,rubio,VE,7.7013,-72.3557,84,
10,puerto santa,PE,-8.9906,-78.6456,89,
11,crane,US,31.3974,-102.3501,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [94]:
# Set parameters to search for a hotel
radius = 10000
params = {"filter":'',"bias":'',"categories":'accommodation.hotel',"apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except(KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
      
# Display sample data
#hotel_df

Starting hotel search
blackmans bay - nearest hotel: Villa Howden
port elizabeth - nearest hotel: Waterford Hotel
rubio - nearest hotel: No hotel found
puerto santa - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
tofol - nearest hotel: Nautilus Resort
hawaiian paradise park - nearest hotel: No hotel found
twon-brass - nearest hotel: KC Hotel
west island - nearest hotel: Cocos Village Bungalows
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
el obeid - nearest hotel: فندق المدينة
santo antonio - nearest hotel: Pousada Onça Puma
morrope - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso
kattivakkam - nearest hotel: No hotel found
weno - nearest hotel: High Tide Hotel
mugia - nearest hotel: Solpor do Coído
enewetak - nearest hotel: No hotel found
impfondo - nearest hotel: Tropicana
kaifeng - nearest hotel: JinjiangInn(Kaifeng)
lazaro cardenas - nearest hotel: Hotel Sol del Pacífic

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [100]:
%%capture --no-display

# Configure the map plot
map2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 700,
    frame_height = 600,
    hover_cols = ["Hotel Name","Country"],
    size = 'Humidity',
    scale = 0.65,
    color = 'City')
# Display the map
map2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)